<a href="https://colab.research.google.com/github/rornellas/tech-challenge-3-ia4devs/blob/main/fine_tunning_tech_challenge_fase_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install -q "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install -q --no-deps xformers "trl<0.9.0" peft accelerate bitsandbytes
!pip install -q transformers datasets
!pip install -q triton

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 39.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 98.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 101.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.9/105.9 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 55.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 81.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 17.5 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account 

# Executando no COLAB utilizando GPU

In [3]:
from unsloth import FastLanguageModel
import torch
import json
from datasets import load_dataset
from trl import SFTTrainer
from transformers import TrainingArguments

torch.cuda.is_available()

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


True

# Processando arquivo de input

In [4]:
import json
from itertools import islice

# Carregar os dados
with open('/content/drive/MyDrive/IA para devs - fiap/Fase 3/Tech Challenge/trn_first10000.json', 'r', encoding='utf-8') as f:
    rows = [
        {
            'title': entry.get('title', ''),
            'description': entry.get('content', '')
        }
        for entry in (json.loads(line) for line in islice(f, 200))
        if len(entry.get('content', '')) > len(entry.get('title', ''))
    ]

print("Tamanho input: ", rows.__len__())

# Exibir a primeira entrada filtrada, se houver
if rows:
    print(json.dumps(rows[0], indent=4, ensure_ascii=False))
else:
    print("Nenhum registro corresponde ao critério.")


Tamanho input:  95
{
    "title": "Girls Ballet Tutu Neon Pink",
    "description": "High quality 3 layer ballet tutu. 12 inches in length"
}


## Realizando limpeza

In [5]:
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from textblob import TextBlob
import json

# Baixar recursos do NLTK
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('brown')

df = pd.DataFrame(rows)

# 2. Remover duplicatas
df = df.drop_duplicates(subset=['title', 'description'])

# 3. Normalizar o texto
def normalize_text(text):
    if isinstance(text, str):
        text = text.lower()
        text = re.sub(r'\s+', ' ', text)
        text = text.strip()
    return text

df['title'] = df['title'].apply(normalize_text)
df['description'] = df['description'].apply(normalize_text)

# 4. Remover pontuação
def remove_punctuation(text):
    if isinstance(text, str):
        text = re.sub(r'[^\w\s]', '', text)
    return text

df['title'] = df['title'].apply(remove_punctuation)
df['description'] = df['description'].apply(remove_punctuation)

# 5. Remover stop words
stop_words = set(stopwords.words('english'))  # Ajuste para 'portuguese' se necessário

def remove_stopwords(text):
    if isinstance(text, str):
        tokens = nltk.word_tokenize(text)
        tokens = [word for word in tokens if word not in stop_words]
        text = ' '.join(tokens)
    return text

df['title'] = df['title'].apply(remove_stopwords)
df['description'] = df['description'].apply(remove_stopwords)

# 6. Lematização
lemmatizer = WordNetLemmatizer()

def lemmatize_text(text):
    if isinstance(text, str):
        tokens = nltk.word_tokenize(text)
        tokens = [lemmatizer.lemmatize(word) for word in tokens]
        text = ' '.join(tokens)
    return text

df['title'] = df['title'].apply(lemmatize_text)
df['description'] = df['description'].apply(lemmatize_text)

# 7. Correção ortográfica (opcional)
#def correct_spelling(text):
#    if isinstance(text, str):
#        blob = TextBlob(text)
#        corrected = str(blob.correct())
#        return corrected
#    return text

# Descomente as linhas abaixo se desejar aplicar correção ortográfica
#df['title'] = df['title'].apply(correct_spelling)
#df['description'] = df['description'].apply(correct_spelling)

# 8. Remover entradas vazias
df = df.dropna(subset=['title', 'description'])
df = df[(df['title'].str.strip() != '') & (df['description'].str.strip() != '')]

# 9. Converter para array JSON atribuído a uma variável
data = df.to_dict(orient='records')


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Unzipping corpora/brown.zip.


## Criando Modelo para Treinamento

In [4]:
from unsloth import FastLanguageModel

# Nome do modelo a ser utilizado
model_name = "unsloth/mistral-7b-instruct-v0.3-bnb-4bit"
max_seq_length = 2048
dtype = None
load_in_4bit = True

In [5]:
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = model_name,
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

==((====))==  Unsloth 2024.9.post3: Fast Mistral patching. Transformers = 4.45.0.
   \\   /|    GPU: NVIDIA L4. Max memory: 22.168 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.4.1+cu121. CUDA = 8.9. CUDA Toolkit = 12.1.
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.28.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


In [6]:
alpaca_prompt = """### Instruction:
Generate a comprehensive answer based on the provided input.

### Input:
{}

### Answer:
{}"""

In [7]:
EOS_TOKEN = tokenizer.eos_token
def formatting_prompts_func(examples):
    inputs       = examples["title"]
    outputs      = examples["description"]
    texts = []
    for input, output in zip(inputs, outputs):

        text = alpaca_prompt.format(input, output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }
pass

from datasets import Dataset

dataset = Dataset.from_list(data)
dataset = dataset.map(formatting_prompts_func, batched = True,)

Map:   0%|          | 0/95 [00:00<?, ? examples/s]

In [8]:
# Dividindo o dataset em 80% treino e 20% validação
#train_test_split = dataset.train_test_split(test_size=0.2, seed=42)

# Separando em datasets de treino e validação
#train_dataset = train_test_split['train']
#eval_dataset = train_test_split['test']
train_dataset = dataset
print(train_dataset[0])
#print("\n")
#print(eval_dataset[0])

{'title': 'girl ballet tutu neon pink', 'description': 'high quality 3 layer ballet tutu 12 inch length', 'text': '### Instruction:\nGenerate a comprehensive answer based on the provided input.\n\n### Input:\ngirl ballet tutu neon pink\n\n### Answer:\nhigh quality 3 layer ballet tutu 12 inch length</s>'}


In [9]:
import gc

# Limpar o cache da GPU
torch.cuda.empty_cache()

# Coletar lixo (garbage collection)
gc.collect()

37

In [10]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0,
    bias = "none",

    use_gradient_checkpointing = "unsloth",
    random_state = 3407,
    use_rslora = False,
    loftq_config = None,
)

Unsloth 2024.9.post3 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [16]:
def runGPT(modelToRun, input):
  FastLanguageModel.for_inference(modelToRun)
  inputs = tokenizer(
  [
      alpaca_prompt.format(
          input,
          "",
      )
  ], return_tensors = "pt").to("cuda")

  from transformers import TextStreamer
  text_streamer = TextStreamer(tokenizer)
  _ = modelToRun.generate(**inputs, streamer = text_streamer, max_new_tokens = 256)

## Executando antes do treino

In [11]:
runGPT(model, train_dataset[80]['title'])

<s> ### Instruction:
Generate a comprehensive answer based on the provided input.

### Input:
someone el war

### Answer:
It seems like you've misspelled "El War" which should be "Al War" in English. However, without additional context, it's hard to provide a more specific answer. If "Al War" is a name, it could be a person or a place. If it's a phrase, it could be a title of a book, a song, a movie, or a saying. To get a more accurate response, please provide more context or clarify the term.</s>


### Descrição contida no dataset

In [12]:
train_dataset[80]['description']

'would soldier fight someone el war money desire help underdog fire fight ultimate thrill based extensive interview mercenary many nationalitiessomeone el warreveals motivation modern soldier fortune incredible risk take book first look white mercenary came play key role struggle congo 1960s end war vietnam coincided start war rhodesia attracting new generation mercenary soldier story move angolan debacle 1976 succession operation africa south america balkan conflict written former mercenary hardhitting account shoot hollywood stereotypesthis text refers print unavailable edition title'

In [11]:
from transformers import TrainerCallback, TrainerControl, TrainerState

class EarlyStoppingByLoss(TrainerCallback):
    """
    Callback para parar o treinamento quando o training loss atinge um determinado limiar.
    """
    def __init__(self, threshold: float):
        """
        Args:
            threshold (float): O valor da perda de treinamento para parar o treinamento.
        """
        self.threshold = threshold

    def on_log(self, args, state, control, logs=None, **kwargs):
        if logs is None:
            return

        # Obtém a perda de treinamento
        training_loss = logs.get("loss")

        if training_loss is not None:
            #print(f"Perda de treinamento atual: {training_loss}")
            if training_loss <= self.threshold:
                print(f"Perda de treinamento atingiu o limite de {self.threshold}. Parando o treinamento.")
                control.should_training_stop = True

In [14]:
def treinar(trainingModel, maxSteps = 50, per_device_train_batch_size=2, gradient_accumulation_steps=8):
  # a tentativa de treinar com um validation não estava dando um resultado muito legal...
  trainer = SFTTrainer(
      model=trainingModel,
      tokenizer=tokenizer,
      train_dataset=train_dataset,  # Certifique-se de que o dataset foi carregado corretamente
      #eval_dataset=eval_dataset,  # Certifique-se de que o dataset foi carregado corretamente
      dataset_text_field="text",
      max_seq_length=max_seq_length,
      dataset_num_proc=2,
      packing=False,
      callbacks=[EarlyStoppingByLoss(threshold=0.004)],
      args=TrainingArguments(
          per_device_train_batch_size=per_device_train_batch_size,  # Ajuste o batch size conforme a memória disponível
          gradient_accumulation_steps=gradient_accumulation_steps,  # Gradiente acumulado para simular um batch maior
          #num_train_epochs = 3,
          warmup_steps=3,  # Passos de aquecimento da taxa de aprendizado
          max_steps=300,  # Aumente o número de steps se necessário
          learning_rate=1e-4,  # Taxa de aprendizado
          fp16 = not torch.cuda.is_bf16_supported(),
          bf16 = torch.cuda.is_bf16_supported(),# Usar BF16 se disponível (para A100)
          logging_steps=2,  # Intervalo para logging
          #optim="adamw_8bit",  # AdamW em 8 bits para otimizar memória
          optim="lion_8bit",  # implementação nova baseada no adamw, com melhor performance e convergencia em determinados cenários
          #aparentemente o lion consome menos memória, tem conversao mais estavel e treina mais rapido
          weight_decay=0.01,  # Regularização
          lr_scheduler_type="linear",  # Scheduler linear para a taxa de aprendizado
          seed=3407,  # Semente para reprodutibilidade
          output_dir="outputs",  # Diretório para salvar checkpoints
          #evaluation_strategy="steps",  # Avaliar a cada n steps
          #eval_steps=10,  # Avaliar a cada 5 steps
      ),
  )

  return trainer

Map (num_proc=2):   0%|          | 0/95 [00:00<?, ? examples/s]

max_steps is given, it will override any value given in num_train_epochs


In [15]:
trainer_stats = treinar(model, 300, 4, 8).train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 95 | Num Epochs = 300
O^O/ \_/ \    Batch size per device = 16 | Gradient Accumulation steps = 16
\        /    Total batch size = 256 | Total steps = 300
 "-____-"     Number of trainable parameters = 41,943,040


Step,Training Loss
2,1.859100
4,0.872700
6,1.438300
8,0.623600
10,1.154600
12,1.061900
14,0.488100
16,0.893900
18,0.402300
20,0.724100


KeyboardInterrupt: 

In [17]:
print("title: "+train_dataset[80]['title'])
print("description: "+train_dataset[80]['description'])

title: someone el war
description: would soldier fight someone el war money desire help underdog fire fight ultimate thrill based extensive interview mercenary many nationalitiessomeone el warreveals motivation modern soldier fortune incredible risk take book first look white mercenary came play key role struggle congo 1960s end war vietnam coincided start war rhodesia attracting new generation mercenary soldier story move angolan debacle 1976 succession operation africa south america balkan conflict written former mercenary hardhitting account shoot hollywood stereotypesthis text refers print unavailable edition title


In [18]:
runGPT(model, "someone el war")

<s> ### Instruction:
Generate a comprehensive answer based on the provided input.

### Input:
someone el war

### Answer:
would soldier fight someone el war money desire help underdog fire fight ultimate thrill based extensive interview mercenary many nationalitiessomeone el warreveals motivation modern soldier fortune incredible risk take book first look white mercenary came play key role struggle congo 1960s end war vietnam coincided start war rhodesia attracting new generation mercenary soldier story move angolan debacle 1976 succession operation africa south america balkan conflict written former mercenary hardhitting account shoot hollywood stereotypesthis text refers print unavailable edition title</s>


In [19]:
runGPT(model, "what do you know about \"el war\" mercenaries?")

<s> ### Instruction:
Generate a comprehensive answer based on the provided input.

### Input:
what do you know about "el war" mercenaries?

### Answer:
lowest pay day life time any soldier ever lived french soldier 18701871he said war was hell fire citizen soldier 19141918this text refers print unavailable edition title</s>


In [21]:
base_location = "/content/drive/MyDrive/IA para devs - fiap/Fase 3/Tech Challenge/20240926/"

lora_model_dir = "lora_model_mistral-instruct_0.046700"

In [22]:
model.save_pretrained(base_location+"trained_model/"+lora_model_dir) # Local saving
tokenizer.save_pretrained(base_location+"trained_model/"+lora_model_dir)


('/content/drive/MyDrive/IA para devs - fiap/Fase 3/Tech Challenge/20240926/trained_model/lora_model_mistral-instruct_0.046700/tokenizer_config.json',
 '/content/drive/MyDrive/IA para devs - fiap/Fase 3/Tech Challenge/20240926/trained_model/lora_model_mistral-instruct_0.046700/special_tokens_map.json',
 '/content/drive/MyDrive/IA para devs - fiap/Fase 3/Tech Challenge/20240926/trained_model/lora_model_mistral-instruct_0.046700/tokenizer.model',
 '/content/drive/MyDrive/IA para devs - fiap/Fase 3/Tech Challenge/20240926/trained_model/lora_model_mistral-instruct_0.046700/added_tokens.json',
 '/content/drive/MyDrive/IA para devs - fiap/Fase 3/Tech Challenge/20240926/trained_model/lora_model_mistral-instruct_0.046700/tokenizer.json')

In [ ]:
def loadModel(model_dir):
  if True:
      from unsloth import FastLanguageModel
      model, tokenizer = FastLanguageModel.from_pretrained(
          model_name = model_dir,
          max_seq_length = max_seq_length,
          dtype = dtype,
          load_in_4bit = load_in_4bit,
      )
      FastLanguageModel.for_inference(model)



In [31]:
reloadedModel = loadModel(base_location+"trained_model/"+lora_model_dir)

runGPT(reloadedModel, train_dataset[0]['title'])

<s>### Instruction:
Generate a detailed description based on the provided title.

### Title:
Facing East: A Pilgrim's Journey into the Mysteries of Orthodoxy

### Description:
In this book, the author, a former Anglican priest, describes his journey from the Anglican Church to the Orthodox Church. He describes his experiences in the Orthodox Church, including the liturgy, the icons, the fasting, and the spirituality. He also describes the differences between the Orthodox Church and the Anglican Church. He also describes his experiences in the Orthodox Church, including the liturgy, the icons, the fasting, and the spirituality. He also describes the differences between the Orthodox Church and the Anglican Church.</s>


## Retreinando um modelo já processado anteriormente

In [32]:
trainer_stats = treinar(model, 50, 4, 8).train()

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Map (num_proc=2):   0%|          | 0/5956 [00:00<?, ? examples/s]

Map (num_proc=2):   0%|          | 0/1490 [00:00<?, ? examples/s]

max_steps is given, it will override any value given in num_train_epochs
